In [5]:
import pandas as pd
from scipy.interpolate import PchipInterpolator
import numpy as np

In [6]:
df = pd.read_csv('BMED_DATA.csv')

df_ag = pd.DataFrame()

# exp 값을 0부터 시작해서 실험별로 연속적으로 증가하도록 매핑
unique_exps = sorted(df['exp'].unique())
exp_map = {old_exp: new_exp for new_exp, old_exp in enumerate(unique_exps)}

for exp_num in unique_exps:
    exp_data = df[df['exp'] == exp_num]

    # For each experiment, generate new time points at 0.25 intervals up to the end time.
    new_times = np.arange(0, exp_data['time'].max() + 0.25, 0.25)

    # The columns to be interpolated
    columns_to_interpolate = ['VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK','CBK', 'I']

    # exp 값을 0부터 시작해서 연속적으로 증가하도록 저장
    temp_df = pd.DataFrame({
        'exp': exp_map[exp_num],
        'V': exp_data['V'].iloc[0],
        'E': exp_data['E'].iloc[0],
        't': new_times
    })

    # apply the monotonic cubic interpolation for each column
    for col in columns_to_interpolate:
        interpolator = PchipInterpolator(exp_data['time'], exp_data[col])
        temp_df[col] = interpolator(new_times)
    
    # add the results to main dataframe
    if df_ag.empty:
        df_ag = temp_df
    else:
        df_ag = pd.concat([df_ag, temp_df], ignore_index=True)

In [7]:
# save CSV file
df_ag.to_csv('BMED_DATA_AG.csv', index=False)

In [8]:
df_ag

,exp,V,E,t,VF,VA,VB,CFLA,CALA,CFK,CBK,I
0,0,20,0.25,0.00,1.000000,1.000000,1.000000,0.495147,0.000518,1.083763,0.011514,0.000000
1,0,20,0.25,0.25,1.008596,0.997544,0.993860,0.490314,0.001626,0.954817,0.136444,0.120313
2,0,20,0.25,0.50,1.016095,0.995401,0.988503,0.485344,0.003696,0.841070,0.246838,0.229167
3,0,20,0.25,0.75,1.021399,0.993886,0.984715,0.480242,0.006447,0.749887,0.335982,0.323438
4,0,20,0.25,1.00,1.023411,0.993311,0.983278,0.475013,0.009596,0.688634,0.397163,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...
587,23,35,0.50,6.00,0.282901,1.443299,2.273113,0.253424,2.204487,0.041429,1.184012,3.404974
588,23,35,0.50,6.25,0.263436,1.443299,2.291290,0.168860,2.222514,0.025455,1.181657,2.937112
589,23,35,0.50,6.50,0.248414,1.443299,2.306515,0.104276,2.235851,0.013400,1.176642,2.319837
590,23,35,0.50,6.75,0.239264,1.443299,2.316642,0.065264,2.243695,0.006190,1.169461,1.588445
